In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[1], 'GPU')
logical_gpus = tf.config.list_logical_devices('GPU')

2025-05-27 03:24:32.324906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748334272.351898   18219 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748334272.360151   18219 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748334272.381432   18219 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748334272.381455   18219 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748334272.381457   18219 computation_placer.cc:177] computation placer alr

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, padding="valid", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

In [4]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [7]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10


I0000 00:00:1748334314.006852   18461 service.cc:152] XLA service 0x2aae7000a090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748334314.006888   18461 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-05-27 03:25:14.048276: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748334314.341478   18461 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-27 03:25:17.092609: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,30,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.2231, f32[32,3,3,3]{3,2,1,0} %bitcast.2238, f32[32]{0} %bitcast.2733), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivat

782/782 - 10s - 13ms/step - accuracy: 0.3851 - loss: 1.6859
Epoch 2/10
782/782 - 2s - 3ms/step - accuracy: 0.5162 - loss: 1.3541
Epoch 3/10
782/782 - 2s - 3ms/step - accuracy: 0.5698 - loss: 1.2147
Epoch 4/10
782/782 - 2s - 3ms/step - accuracy: 0.6064 - loss: 1.1175
Epoch 5/10
782/782 - 2s - 3ms/step - accuracy: 0.6361 - loss: 1.0404
Epoch 6/10
782/782 - 2s - 3ms/step - accuracy: 0.6612 - loss: 0.9769
Epoch 7/10
782/782 - 2s - 3ms/step - accuracy: 0.6789 - loss: 0.9262
Epoch 8/10
782/782 - 2s - 3ms/step - accuracy: 0.6927 - loss: 0.8809
Epoch 9/10
782/782 - 2s - 3ms/step - accuracy: 0.7094 - loss: 0.8425
Epoch 10/10
782/782 - 2s - 3ms/step - accuracy: 0.7231 - loss: 0.8030


2025-05-27 03:25:44.258947: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,30,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.526, f32[32,3,3,3]{3,2,1,0} %bitcast.533, f32[32]{0} %bitcast.535), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1/convolution" source_file="/project/nanocourse/SWE_OOP/train03/miniconda3/envs/ml/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-27 03:25:44.268270: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Om

157/157 - 1s - 9ms/step - accuracy: 0.6896 - loss: 0.9001


[0.9000521302223206, 0.6895999908447266]

In [9]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


modelFunc = my_model()
modelFunc.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [10]:
print(modelFunc.summary())

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,930 (882.54 KB)

 Trainable params: 225,482 (880.79 KB)

 Non-trainable params: 448 (1.75 KB)

None


In [11]:
modelFunc.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
modelFunc.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 9s - 11ms/step - accuracy: 0.5208 - loss: 1.3455
Epoch 2/10
782/782 - 3s - 3ms/step - accuracy: 0.6583 - loss: 0.9810
Epoch 3/10
782/782 - 3s - 3ms/step - accuracy: 0.7128 - loss: 0.8271
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.7516 - loss: 0.7150
Epoch 5/10
782/782 - 3s - 3ms/step - accuracy: 0.7807 - loss: 0.6335
Epoch 6/10
782/782 - 3s - 3ms/step - accuracy: 0.8067 - loss: 0.5625
Epoch 7/10
782/782 - 3s - 3ms/step - accuracy: 0.8280 - loss: 0.4996
Epoch 8/10
782/782 - 3s - 3ms/step - accuracy: 0.8489 - loss: 0.4425
Epoch 9/10
782/782 - 3s - 3ms/step - accuracy: 0.8684 - loss: 0.3915
Epoch 10/10
782/782 - 3s - 3ms/step - accuracy: 0.8841 - loss: 0.3459
157/157 - 2s - 10ms/step - accuracy: 0.6931 - loss: 1.0485


[1.0485016107559204, 0.6930999755859375]

In [ ]:
# To Do:
# 1. construct a new class (type) called MyModelSubClass which is a keras.Model, instantiate it, 
# and then fit the model to the data. 
#
# 2. Now explore: See what kind of accuracy you can get by training longer, 
# increasing model size, changing the kernel sizes.


In [40]:
class MyModelSubClass(keras.Model):
    def __init__(self,name='MyModelSubclass', **kwargs):
        # call super (base) class's init
        super(MyModelSubClass, self).__init__(name=name, **kwargs)
        
        # define layers to be used in forward pass
        self.conv1 = self.conv_block(32)
        self.conv2 = self.conv_block(64)
        self.conv3 = self.conv_block(128)

        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation="relu")
        self.out = layers.Dense(10)

    def conv_block(self, numConv):
        return keras.Sequential([
            layers.Conv2D(numConv, 3, padding='valid'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling2D()
        ])
        
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.out(x)
        # return the prediction
        return x
    
    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [41]:
# Instantiate the model
model = MyModelSubClass()

In [42]:
# Print a summary of the model
dummy_input = keras.Input(shape=(32, 32, 3))
_ = model(dummy_input) 

model.summary()

Model: "MyModelSubclass"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_31 (Sequential)      │ (None, 15, 15, 32)     │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_32 (Sequential)      │ (None, 6, 6, 64)       │        18,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_33 (Sequential)      │ (None, 2, 2, 128)      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,626 (498.54 KB)

 Trainable params: 127,178 (496.79 KB)

 Non-trainable params: 448 (1.75 KB)

In [43]:
# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [44]:
# Fit the model
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 9s - 11ms/step - accuracy: 0.5093 - loss: 1.3795
Epoch 2/10
782/782 - 3s - 3ms/step - accuracy: 0.6461 - loss: 1.0143
Epoch 3/10
782/782 - 3s - 3ms/step - accuracy: 0.6986 - loss: 0.8747
Epoch 4/10
782/782 - 3s - 3ms/step - accuracy: 0.7341 - loss: 0.7736
Epoch 5/10
782/782 - 3s - 3ms/step - accuracy: 0.7611 - loss: 0.6968
Epoch 6/10
782/782 - 3s - 3ms/step - accuracy: 0.7833 - loss: 0.6304
Epoch 7/10
782/782 - 3s - 3ms/step - accuracy: 0.8028 - loss: 0.5715
Epoch 8/10
782/782 - 3s - 3ms/step - accuracy: 0.8226 - loss: 0.5180
Epoch 9/10
782/782 - 3s - 3ms/step - accuracy: 0.8391 - loss: 0.4748
Epoch 10/10
782/782 - 3s - 3ms/step - accuracy: 0.8545 - loss: 0.4296
157/157 - 2s - 10ms/step - accuracy: 0.6896 - loss: 0.9732


[0.9731577038764954, 0.6895999908447266]

In [45]:
# Check your model's performance
# training: 0.8545, testing: 0.6896

In [ ]:
### Rinse and repeat; this time add more layers. 

In [55]:
class MyModelSubClass_Layers(keras.Model):
    def __init__(self,name='MyModelSubclass_Layers', **kwargs):
        # call super (base) class's init
        super(MyModelSubClass_Layers, self).__init__(name=name, **kwargs)
        
        # define layers to be used in forward pass
        self.conv1 = self.conv_block(32)
        self.conv2 = self.conv_block(64)
        self.conv3 = self.conv_block(128)
        self.conv4 = self.conv_block(256)

        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation="relu")
        self.out = layers.Dense(10)

    def conv_block(self, numConv):
        return keras.Sequential([
            layers.Conv2D(numConv, 3, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling2D()
        ])
        
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.out(x)
        # return the prediction
        return x
    
    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [56]:
# Instantiate the model
model1 = MyModelSubClass_Layers()

# Print a summary of the model
dummy_input = keras.Input(shape=(32, 32, 3))
_ = model1(dummy_input) 

model1.summary()

Model: "MyModelSubclass_Layers"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_49 (Sequential)      │ (None, 16, 16, 32)     │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_50 (Sequential)      │ (None, 8, 8, 64)       │        18,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_51 (Sequential)      │ (None, 4, 4, 128)      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_52 (Sequential)      │ (None, 2, 2, 256)      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,586 (1.74 MB)

 Trainable params: 455,626 (1.74 MB)

 Non-trainable params: 960 (3.75 KB)

In [57]:
# Compile the model
model1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

# Fit the model
model1.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model1.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10


2025-05-27 04:05:11.998200: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.12 = (f32[64,32,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.4614, f32[32,3,3,3]{3,2,1,0} %bitcast.4621, f32[32]{0} %bitcast.5814), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="MyModelSubclass_Layers_1/sequential_49_1/conv2d_60_1/convolution" source_file="/project/nanocourse/SWE_OOP/train03/miniconda3/envs/ml/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-27 04:05:12.007639: I external/local_xla/xla/service/

782/782 - 12s - 15ms/step - accuracy: 0.5616 - loss: 1.2287
Epoch 2/10
782/782 - 3s - 4ms/step - accuracy: 0.7051 - loss: 0.8434
Epoch 3/10
782/782 - 3s - 4ms/step - accuracy: 0.7634 - loss: 0.6832
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.8053 - loss: 0.5583
Epoch 5/10
782/782 - 3s - 4ms/step - accuracy: 0.8414 - loss: 0.4627
Epoch 6/10
782/782 - 3s - 4ms/step - accuracy: 0.8768 - loss: 0.3684
Epoch 7/10
782/782 - 3s - 4ms/step - accuracy: 0.9022 - loss: 0.2928
Epoch 8/10
782/782 - 3s - 4ms/step - accuracy: 0.9258 - loss: 0.2285
Epoch 9/10
782/782 - 3s - 4ms/step - accuracy: 0.9421 - loss: 0.1813
Epoch 10/10
782/782 - 3s - 4ms/step - accuracy: 0.9588 - loss: 0.1353
157/157 - 2s - 11ms/step - accuracy: 0.7389 - loss: 0.9420


[0.9419621229171753, 0.7389000058174133]

In [ ]:
# Check your model's performance. How does the performance compare with the smaller model?
# It's improving, the training dataset has improved well but the accuracy for testing dataset is still low.

In [ ]:
### Rinse and repeat; this time change the kernel size. 

In [67]:
class MyModelSubClass_Kernel(keras.Model):
    def __init__(self,name='MyModelSubClass_Kernel', **kwargs):
        # call super (base) class's init
        super(MyModelSubClass_Kernel, self).__init__(name=name, **kwargs)
        
        # define layers to be used in forward pass
        self.conv1 = self.conv_block(32)
        self.conv2 = self.conv_block(64)
        self.conv3 = self.conv_block(128)

        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation="relu")
        self.out = layers.Dense(10)

    def conv_block(self, numConv):
        return keras.Sequential([
            layers.Conv2D(numConv, 5, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling2D()
        ])
        
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.out(x)
        # return the prediction
        return x
    
    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [68]:
# Instantiate the model
model1 = MyModelSubClass_Kernel()

# Print a summary of the model
dummy_input = keras.Input(shape=(32, 32, 3))
_ = model1(dummy_input) 

model1.summary()

Model: "MyModelSubClass_Kernel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_62 (Sequential)      │ (None, 16, 16, 32)     │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_63 (Sequential)      │ (None, 8, 8, 64)       │        51,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_64 (Sequential)      │ (None, 4, 4, 128)      │       205,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 391,306 (1.49 MB)

 Trainable params: 390,858 (1.49 MB)

 Non-trainable params: 448 (1.75 KB)

In [69]:
# Compile the model
model1.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

# Fit the model
model1.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model1.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10


2025-05-27 04:16:59.621805: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.3714, f32[32,3,5,5]{3,2,1,0} %bitcast.3721, f32[32]{0} %bitcast.4674), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="MyModelSubClass_Kernel_1/sequential_62_1/conv2d_73_1/convolution" source_file="/project/nanocourse/SWE_OOP/train03/miniconda3/envs/ml/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-27 04:16:59.626277: I external/local_xla/xla/service/g

782/782 - 11s - 14ms/step - accuracy: 0.5160 - loss: 1.3612
Epoch 2/10
782/782 - 4s - 5ms/step - accuracy: 0.6674 - loss: 0.9522
Epoch 3/10
782/782 - 4s - 5ms/step - accuracy: 0.7259 - loss: 0.7899
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.7647 - loss: 0.6829
Epoch 5/10
782/782 - 4s - 5ms/step - accuracy: 0.7967 - loss: 0.5887
Epoch 6/10
782/782 - 4s - 5ms/step - accuracy: 0.8229 - loss: 0.5139
Epoch 7/10
782/782 - 4s - 4ms/step - accuracy: 0.8474 - loss: 0.4438
Epoch 8/10
782/782 - 3s - 4ms/step - accuracy: 0.8706 - loss: 0.3789
Epoch 9/10
782/782 - 4s - 4ms/step - accuracy: 0.8952 - loss: 0.3121
Epoch 10/10
782/782 - 4s - 5ms/step - accuracy: 0.9161 - loss: 0.2567
157/157 - 2s - 13ms/step - accuracy: 0.7436 - loss: 0.8315


[0.8314716815948486, 0.7436000108718872]

In [ ]:
# Check your model's performance. How does the performance compare with the other two models?
# Adding layers and kernels both improve the perfomances. But the training dataset performance is still a lot better than testing datasets.